In [54]:
import glob
import pandas as pd
import math

In [4]:
train_set_dir = "/media/data2/Data/wsdm2019/data/training_set/"
second_stage_file_dir = "/media/data2/Data/wsdm2019/python/data/second_stage/"

second_stage_files = sorted(glob.glob(second_stage_file_dir+"*.csv"))

In [115]:
test_file = second_stage_files[0]

In [116]:
test_file

'/media/data2/Data/wsdm2019/python/data/second_stage/log_9_20180731_000000000000_second_stage.csv'

In [114]:
test_file[52:]

'log_9_20180731_000000000000_second_stage.csv'

In [28]:
original_file = train_set_dir + test_file[52:-17] + ".csv"

In [29]:
original_file

'/media/data2/Data/wsdm2019/data/training_set/log_9_20180731_000000000000.csv'

In [30]:
train_f = pd.read_csv(original_file)

In [31]:
logit_f = pd.read_csv(test_file)

In [34]:
train_f['logits'] = logit_f['logits']

In [37]:
test_f = train_f[['session_id', 'skip_2', 'logits']]

In [48]:
test_groups = test_f.groupby('session_id')

In [108]:
predicted = []
groundtruth = []
for name, group in test_groups:
    cut_value = math.floor(len(group) / 2)
    group_logits = (group['logits'].values[cut_value:] > 0.5).astype(int).tolist()
    group_skip_2 = group['skip_2'].values[cut_value:].tolist()
    
    groundtruth.append(group_skip_2)
    predicted.append(group_logits)

In [111]:
print(evaluate(predicted, groundtruth))

(0.6292740926015201, 0.8006008241942906)


In [50]:
def evaluate(submission, groundtruth):
    ap_sum = 0.0
    first_pred_acc_sum = 0.0
    counter = 0
    for sub, tru in zip(submission, groundtruth):
        if len(sub) != len(tru):
            raise Exception('Line {} should contain {} predictions, but instead contains '
                            '{}'.format(counter+1,len(tru),len(sub)))
        ap_sum += ave_pre(sub,tru,counter)
        first_pred_acc_sum += sub[0] == tru[0]
        counter+=1
    ap = ap_sum/counter
    first_pred_acc = first_pred_acc_sum/counter
    return ap, first_pred_acc

def ave_pre(submission,groundtruth,counter):
    s = 0.0
    t = 0.0
    c = 1.0
    for x, y in zip(submission, groundtruth):
        if x != 0 and x != 1:
            raise Exception('Invalid prediction in line {}, should be 0 or 1'.format(counter))
        if x==y:
            s += 1.0
            t += s / c
        c += 1
    return t/len(groundtruth)